#### AMPEL intro

AMPEL is software framework designed for processing heterogeneous streamed data. 

AMPEL was not developed to provide a specific scientific resource, but rather an environment where it is easy to ensure that a scientific program fulfills the strict requirement of the next generation real-time experiments: efficient and powerful analysis, where provenance and reproducibiltiy is paramount. In particular, to guarantee the last point requires algorithms (which make real-time deicsions) be separated from infrastructure (which will likely evolve with time and project phase).

An AMPEL _user_ constructs a configuration file which describes every step of how an incoming alert stream should be processed. This can be broken down into selecting which _units_ should be executed, and which _parameters_ each of these should be provided. An AMPEL _live instance_ executes these units, based on the input data, as requested and stores all intermediate and final data in a databse. 

Provenance/reproducibility is ensured through multiple layers. First, each live instance is run from a container which can be retrieved later and together with a data archive replay the full stream. Second, AMPEL contains an extensive set of logs and a transient-specific _Journal_ which details all related actions/decisions. Finally, each unit and channel configuration file is drawn from a specific (tagged) github version. 

The series of notebooks provided here gradually builds toward a sample full configration. 

#### Sample science case

Each AMPEl _channel_ is designed with a science goal (or "hypothesis/test") in mind. A much discussed current topic is the origin of the extragalactic neutrino flux observed e.g. by IceCube, with one of the potential sources being supernovae interacting with circumstellar material (SNIIn). We here wish to investigate whether a particular subtype of these, SN2009ip-like SNe with recent previous outbursts, are regularly found within the uncertainty region of neutrino alerts. 

The steps for this science program would be: Identify transients with optical lightcurves compatible with SN2009ip AND which coincide with neutrino alerts. For such targets, obtain follow-up spectroscopy to confirm classification (i.e. an external reaction). 

#### This notebook - Tutorial 3

This notebooks mimics one approach for how multi-messenger data can be combined with the ZTF optical stream. 

A unit - `T2MultiMessMatch` - retrieves a recent list of potential multi-messenger events, together with information of their spatial, temporal and energetics properties. It will then return a combined "p-value" of how closely these match the properties of the optical transients. A set of scaling factors determine the relative weight between these dimensions. A real unit would read this information from a local mirror of e.g. LIGO/VIRGO or IceCube events, while we here use an invented event.

Path to unit: `Ampel-contrib-sample/ampel/contrib/sample/t2/T2MultiMessMatch.py`

The final step of this notebook is the selection of events we consider "good" matches, based on having a small p-value.

As in Tutorial 2, this notebook thus assumes a mongod instance to be running and accessible through 27017. (The port can be changed through the mongo key of the ampel_config.yml file).

This is an example of "non-native" multi-messenger matching: a main stream is being compared with an (at any given point in time) fixed set of external regions. AMPEL can also carry out "native" multi-messenger programs, where non-heterogeneous streams are ingested into the system. This adds analysis power, but adds the complexity of creating transient states in the T1 AMPEL tier. 


In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext AmpelLogger T2Processor T3Processor ChannelModel AlertProcessor TarAlertLoader ChannelModel AbsAlertFilter T2MultiMessMatch

In [ ]:
AMPEL_CONF = "../../ampel_config.yml"
ALERT_ARCHIVE = '../sample_data/ztfpub_200917_pruned.tar.gz'

In [ ]:
# The operation context is created based on a setup configuration file.
# db_prefix sets the DB name to use
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTutorial",
    purge_db = True,
)

In [ ]:
# Our key tool will be a skeleton multi-messenger matching unit which determines spatial, temporal and energetic
# differences. The dummy unit uses a fixed entry corresponding to a MM region, a live example
# instead uses e.g LIGO/VIRGO or IceCube contours.
T2MultiMessMatch??

In [ ]:
# The Unit parameters will define how 
mm_match_conf = {
    'temporal_pull_scaling' : 1,         # Neutral - we do not know when Neutrinos are emitted
    'spatial_pull_scaling'  : 3.,        # Reasonably sure regarding location
    'energy_pull_scaling'   : 0.001,     # Little constraint on energy, deweight this
    'match_where'           : 'latest',  # latest, first or mean
}
mm_match_config_id = ctx.add_config_id( mm_match_conf )

In [ ]:
# A scientific program, a channel, is added
ctx.add_channel(
    name="demo_SN09if",
    access=['ZTF', 'ZTF_PUB']
)

In [ ]:
# The channel is constructed from two units, each controlled by parameters. 
# Lets start with the straightforward filter 
filter_conf = {
    'min_rb':0.3,
    'min_ndet':7,
    'min_tspan':10,
    'max_tspan' : 200,
    'min_gal_lat':15,
}
filter_config_id = ctx.add_config_id( filter_conf )

In [ ]:
# The template matching has now been moved into a separate unit:
# T2SNcosmoComp
# where we added some configurability. 
match_conf = {
    'target_model_name':'v19-2009ip-corr', 
    'base_model_name':'salt2', 
    'chi2dof_cut':2.,
    'chicomp_scaling':0.5,
}
match_config_id = ctx.add_config_id( match_conf )

In [ ]:
# A channel can specify which streams to read, how these should be combined and what units
# should be run on each data combination.
# This is provided as directives to the AlertProcessor, which besides processing the alerts
# also submit tickets to the DB concerning further operations to execute for any transients
# that pass the initial filter stage.
ap = AlertProcessor(
    context = ctx,
    process_name = "ipyton_notebook_test",
    supplier = "ZiAlertSupplier",
    log_profile = "debug",
    directives = [
        {
            "channel": "demo_SN09if", 
            "filter": {"unit": "SimpleDecentFilterCopy","config": filter_config_id
                        },
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                {'unit': 'T2SNcosmoComp',
                                 'config': match_config_id
                                },
                                {'unit': 'T2MultiMessMatch',
                                 'config': mm_match_config_id
                                },
                                
                            ]
                        }
                    }
                ],
            }
        }
    ]
)

In [ ]:
# Provide a link to the alert collection to use
ap.set_loader(TarAlertLoader(file_path=ALERT_ARCHIVE))

In [ ]:
ap.set_iter_max(1000)

In [ ]:
ap.run()

In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="debug")

In [ ]:
t2p.run()

So far an input data stream has been filtered, and some sort of calculation has been done on the accepted sample. The next step for a full channel is usually some sort of _reaction_. These can vary between sending immediate alarms (e.g. through Slack or GCN), triggering follow-up observations or propagating information (e.g. for inspection in a frontent such as SkyPortal). 

Such reactions take place in the T3 tier. A simple `T3HelloWorld` unit is used, but the  `react` method can be configured to do most other things. Sample T3 units react with TNS, Slack, Dropbox, SkyPortal and GCN.

Key steps of configuring the T3 procss comes through the `selection` directive, where we select tansients that produced the required target match, and the `execute` directive which regulates which T3 units are run. 

In [ ]:
# Select transients with ok match
t3 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "demo_SN09if",
    directives = [ {
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                't2_filter':  {
                    'unit': 'T2SNcosmoComp',
                    'match': {'target_match': True}
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "demo_SN09if"
                                }
                            },
                            "execute": [
                                {
                                    "unit": "T3HelloWorld",
                                    "config": {
                                        't2info_from' : ['T2SNcosmoComp', 'T2MultiMessMatch']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
t3.run()

In deciding which targets to follow we wish to also make use of the match criteria. Can be done as follows:

In [ ]:
t3 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "demo_SN09if",
    directives = [ {
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                't2_filter': { 
                    'all_of': [                
                        {
                            'unit': 'T2SNcosmoComp',
                            'match': {'target_match': True}
                        }, 
                        {
                            'unit': 'T2MultiMessMatch',
                            'match': {'best_match': {"$lt":1} }
                        },                         
                    ]
            } }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "demo_SN09if"
                                }
                            },
                            "execute": [
                                {
                                    "unit": "T3HelloWorld",
                                    "config": {
                                        't2info_from' : ['T2SNcosmoComp', 'T2MultiMessMatch']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)

In [ ]:
t3.run()

Looks like ZTF20abyfpze is our target! Next step would be to propagate the coordinates to a suitable facility (or export for visual inspection).